In [1]:
import scanpy as sc
import numpy as np
data_path = '../dataset/Chromium_FFPE_Human_Breast_Cancer_Chromium_FFPE_Human_Breast_Cancer_count_sample_filtered_feature_bc_matrix.h5'
sc_data = sc.read_10x_h5(data_path)
sc_data.X = sc_data.X.toarray()
# Randomly sample 80,000 cells
n_cells = sc_data.shape[0]
sample_size = min(80000, n_cells)  # Ensure we don't try to sample more than available cells
random_indices = np.random.choice(n_cells, size=sample_size, replace=False)

# Create a new AnnData object with the sampled cells
sc_data = sc_data[random_indices, :]

# Print to confirm the shape
print(sc_data)

/home/hanchuangyi/miniconda3/envs/tangram/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/hanchuangyi/miniconda3/envs/tangram/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


View of AnnData object with n_obs × n_vars = 30365 × 18082
    var: 'gene_ids', 'feature_types', 'genome'


In [2]:
st_data_path = '../dataset/Xenium_breast_cancer_sample1_replicate1.h5ad'
st_data = sc.read_h5ad(st_data_path)
st_data

AnnData object with n_obs × n_vars = 167780 × 313
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
    var: 'gene_ids', 'feature_types', 'genome'
    obsm: 'spatial'

In [3]:
# 检查 sc_data.var_names 是否有重复项
duplicates = sc_data.var_names[sc_data.var_names.duplicated()]
print(f"重复的基因: {duplicates}")
# 删除 sc_data.var_names 中的重复基因
sc_data = sc_data[:, ~sc_data.var_names.duplicated()]

重复的基因: Index(['TBCE', 'HSPA14', 'TMSB15B'], dtype='object')


In [4]:
st_data.X = st_data.X.toarray()

In [ ]:
import tangram as tg
tg.pp_adatas(sc_data, st_data, genes=None)
ad_map = tg.map_cells_to_space(sc_data, st_data)

INFO:root:306 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:306 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 306 genes and None density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.311
Score: 0.715
Score: 0.767
Score: 0.786
Score: 0.796
Score: 0.803
Score: 0.809
Score: 0.812


In [ ]:
ad_ge = tg.project_genes(ad_map, sc_data)

In [ ]:
imputed_st_data_path = '../dataset/ad_ge_imputed_Xenium_breast_cancer_sample1_replicate1.h5ad'
imputed_intermediate_st_data_path = '../dataset/ad_map_imputed_Xenium_breast_cancer_sample1_replicate1.h5ad'
ad_ge.write_h5ad(imputed_st_data_path)
ad_map.write_h5ad(imputed_intermediate_st_data_path)

In [2]:
imputed_st_data_path = '../dataset/ad_ge_imputed_Xenium_breast_cancer_sample1_replicate1.h5ad'
import scanpy as sc
adata = sc.read_h5ad(imputed_st_data_path)
adata

AnnData object with n_obs × n_vars = 167780 × 17693
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'uniform_density', 'rna_count_based_density'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'sparsity', 'is_training'
    uns: 'overlap_genes', 'training_genes'

In [9]:
adata.obsm['spatial'] = original_adata.obsm['spatial'].copy()

In [10]:
adata.write_h5ad(imputed_st_data_path)

In [6]:
original_adata = sc.read_h5ad('../dataset/Xenium_breast_cancer_sample1_replicate1.h5ad')
original_adata

AnnData object with n_obs × n_vars = 167780 × 313
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
    var: 'gene_ids', 'feature_types', 'genome'
    obsm: 'spatial'

In [8]:

# 比较两列是否逐行相同
comparison_result = original_adata.obs['x_centroid'].equals(adata.obs['x_centroid'])

# 输出比较结果
if comparison_result:
    print("两列完全相同")
else:
    print("两列不相同")

# 找出不相同的行
difference = original_adata.obs['x_centroid'] != adata.obs['x_centroid']

# 打印不相同的行
print(original_adata.obs.loc[difference, 'x_centroid'])
print(adata.obs.loc[difference, 'x_centroid'])


两列完全相同
Series([], Name: x_centroid, dtype: float64)
Series([], Name: x_centroid, dtype: float64)


In [5]:
import pandas as pd
pd.set_option('display.max_rows',None)
adata.var.head(200)

,gene_ids,feature_types,genome,n_cells,sparsity,is_training
samd11,ENSG00000187634,Gene Expression,GRCh38,6500,0.785938,False
noc2l,ENSG00000188976,Gene Expression,GRCh38,8285,0.727153,False
klhl17,ENSG00000187961,Gene Expression,GRCh38,5265,0.826610,False
plekhn1,ENSG00000187583,Gene Expression,GRCh38,5102,0.831978,False
perm1,ENSG00000187642,Gene Expression,GRCh38,436,0.985641,False
hes4,ENSG00000188290,Gene Expression,GRCh38,9740,0.679236,False
isg15,ENSG00000187608,Gene Expression,GRCh38,9458,0.688523,False
agrn,ENSG00000188157,Gene Expression,GRCh38,12888,0.575564,False
rnf223,ENSG00000237330,Gene Expression,GRCh38,447,0.985279,False
c1orf159,ENSG00000131591,Gene Expression,GRCh38,2713,0.910654,False
